In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive')

In [ ]:
import pandas as pd
import tensorflow as tf

In [ ]:
df = pd.read_csv('dataset_annotations.csv')

In [ ]:
df.tail(10)

,idx,label,sentence
792,792,1,When does the train usually leave.
793,793,1,When does the train usually leave.
794,794,1,She always forgets her purse.
795,795,0,She always forget her purse.
796,796,1,He drives to work.
797,797,0,He drive to work.
798,798,1,He drives to work.
799,799,1,He drives to work.
800,800,1,She thinks he is very handsome.
801,801,0,She think he is very handsome.


In [ ]:
len(df)

802

In [ ]:
pip install pandas-tfrecords

In [ ]:
from pandas_tfrecords import pd2tf, tf2pd

pd2tf(df, './tfrecords_forder1')
my_df = tf2pd('./tfrecords_forder1')

In [ ]:
df

,idx,label,sentence
0,0,1,Anthony is throwing the football.
1,1,0,Anthony is throw the football.
2,2,1,She accepted the job offer.
3,3,0,She accept the job offer.
4,4,1,He thought about his stupid mistake in the test.
...,...,...,...
797,797,0,He drive to work.
798,798,1,He drives to work.
799,799,1,He drives to work.
800,800,1,She thinks he is very handsome


In [ ]:
type(df)

pandas.core.frame.DataFrame

In [ ]:
custome_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(df[['sentence']].values, tf.string),
            tf.cast(df[['idx', 'label']].values, tf.int32)
        )
    )
)

for features_tensor, target_tensor in custome_dataset:
    print(f'features:{features_tensor} target:{target_tensor}')
    break

features:[b'Anthony is throwing the football.'] target:[0 1]


In [ ]:
tf.data.experimental.save(custome_dataset,'/content/drive/MyDrive')

In [ ]:
print("Installing dependencies...")
%tensorflow_version 2.x
!pip install -q t5

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)



import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

import t5
import t5.models
import seqio

BASE_DIR = "gs://namratacomp8240" #@param { type: "string" }
if not BASE_DIR or BASE_DIR == "gs://":
  raise ValueError("You must enter a BASE_DIR.")
DATA_DIR = os.path.join(BASE_DIR, "data")
MODELS_DIR = os.path.join(BASE_DIR, "models")
ON_CLOUD = True


if ON_CLOUD:
  print("Setting up GCS access...")
  import tensorflow_gcs_config
  from google.colab import auth
  # Set credentials for GCS reading/writing from Colab and TPU.
  TPU_TOPOLOGY = "v2-8"
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    TPU_ADDRESS = tpu.get_master()
    print('Running on TPU:', TPU_ADDRESS)
  except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
  auth.authenticate_user()
  tf.enable_eager_execution()
  tf.config.experimental_connect_to_host(TPU_ADDRESS)
  tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

Installing dependencies...
Setting up GCS access...
Running on TPU: grpc://10.72.96.82:8470
Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
ds = tfds.load(
    "glue/cola",
    data_dir=DATA_DIR,
    # Download data locally for preprocessing to avoid using GCS space.
    download_and_prepare_kwargs={"download_dir": "./glue"})

INFO:absl:Load dataset info from gs://namratacomp8240/data/glue/cola/2.0.0
INFO:absl:Reusing dataset glue (gs://namratacomp8240/data/glue/cola/2.0.0)
INFO:absl:Constructing tf.data.Dataset glue for split None, from gs://namratacomp8240/data/glue/cola/2.0.0


In [ ]:
print("A few raw validation examples...")
for ex in tfds.as_numpy(ds["validation"].take(2)):
  print(ex)

A few raw validation examples...
{'idx': 0, 'label': 1, 'sentence': b'Anthony is throwing the football.'}
{'idx': 1, 'label': 0, 'sentence': b'Anthony is throw the football.'}


In [ ]:
def get_config(name):
  for b in tfds.text.glue.Glue.builder_configs.values():
    if b.name == name:
      return b

config = get_config('cola')

In [ ]:
import seqio
import t5.data
from t5.data import postprocessors
from t5.data import preprocessors
from t5.data.glue_utils import get_glue_metric
from t5.data.glue_utils import get_glue_postprocess_fn
from t5.data.glue_utils import get_glue_text_preprocessor
from t5.data.glue_utils import get_super_glue_metric
from t5.evaluation import metrics
import tensorflow_datasets as tfds

DEFAULT_OUTPUT_FEATURES = {
    "inputs": seqio.Feature(
        vocabulary=t5.data.get_default_vocabulary(), add_eos=True,
        required=False),
    "targets": seqio.Feature(
        vocabulary=t5.data.get_default_vocabulary(), add_eos=True)
}


seqio.TaskRegistry.add(
  "all_mix",
  source=seqio.TfdsDataSource(
      tfds_name="glue/cola:2.0.0",
      tfds_data_dir=DATA_DIR),
  preprocessors=[
  get_glue_text_preprocessor(config),
  seqio.preprocessors.tokenize,
  seqio.CacheDatasetPlaceholder(),
  seqio.preprocessors.append_eos_after_trim,
      ],
metric_fns=get_glue_metric(config.name),
output_features=DEFAULT_OUTPUT_FEATURES,
postprocess_fn=get_glue_postprocess_fn(config))

In [ ]:
import os


MODEL_SIZE = "small" #@param["small", "base", "large", "3B", "11B"]
# Public GCS path for T5 pre-trained model checkpoints
BASE_PRETRAINED_DIR = "gs://t5-data/pretrained_models"
PRETRAINED_DIR = os.path.join(BASE_PRETRAINED_DIR, MODEL_SIZE)
MODEL_DIR = os.path.join(MODELS_DIR, MODEL_SIZE)

if ON_CLOUD and MODEL_SIZE == "3B":
  tf.logging.warning(
      "The `3B` model is too large to use with the 5GB GCS free tier. "
      "Make sure you have at least 25GB on GCS before continuing."
  )
elif ON_CLOUD and MODEL_SIZE == "11B":
  raise ValueError(
      "The `11B` parameter is too large to fine-tune on the `v2-8` TPU "
      "provided by Colab. Please comment out this Error if you're running "
      "on a larger TPU."
  )

# Set parallelism and batch size to fit on v2-8 TPU (if possible).
# Limit number of checkpoints to fit within 5GB (if possible).
model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)
# The models from our paper are based on the Mesh Tensorflow Transformer.
model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    sequence_length={"inputs": 128, "targets": 32},
    learning_rate_schedule=0.003,
    save_checkpoints_steps=5000,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None,
    iterations_per_loop=100,
)

In [ ]:
FINETUNE_STEPS =  1000#@param {type: "integer"}

model.finetune(
    mixture_or_task_name="all_mix",
    pretrained_model_dir=PRETRAINED_DIR,
    finetune_steps=FINETUNE_STEPS
)

INFO:root:system_path_file_exists:gs://t5-data/pretrained_models/small/operative_config.gin
ERROR:root:Path not found: gs://t5-data/pretrained_models/small/operative_config.gin
INFO:root:Skipping import of unknown module `t5.data.sentencepiece_vocabulary` (skip_unknown=True).


INFO:tensorflow:Using config: {'_model_dir': 'gs://namratacomp8240/models/small', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.72.96.82:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.72.96.82:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.72.96.82:8470', '_evaluation_master': 'grpc://10.72.96.82:8470', '_is_chief':

In [ ]:
# Use a larger batch size for evaluation, which requires less memory.
model.batch_size = train_batch_size * 4
model.eval(
    mixture_or_task_name="all_mix",
    checkpoint_steps="all"
)

INFO:root:system_path_file_exists:gs://namratacomp8240/models/small/operative_config.gin
ERROR:root:Path not found: gs://namratacomp8240/models/small/operative_config.gin
INFO:absl:Load dataset info from gs://namratacomp8240/data/glue/cola/2.0.0
INFO:absl:Adding task 'all_mix' with predict metric_fn(s).
INFO:absl:Load dataset info from gs://namratacomp8240/data/glue/cola/2.0.0
INFO:absl:Reusing dataset glue (gs://namratacomp8240/data/glue/cola/2.0.0)
INFO:absl:Constructing tf.data.Dataset glue for split validation, from gs://namratacomp8240/data/glue/cola/2.0.0
INFO:absl:Automatically caching small dataset in memory: 'all_mix:validation'
INFO:absl:Skipping packing/padding for 'all_mix' since sequence length is None.
INFO:absl:Setting sequence lengths to {'inputs': 28, 'targets': 2}
INFO:absl:Evaluating checkpoint step: 1000000


INFO:tensorflow:Using config: {'_model_dir': 'gs://namratacomp8240/models/small', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.72.96.82:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.72.96.82:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.72.96.82:8470', '_evaluation_master': 'grpc://10.72.96.82:8470', '_is_chief':

INFO:absl:Load dataset info from gs://namratacomp8240/data/glue/cola/2.0.0
INFO:absl:Reusing dataset glue (gs://namratacomp8240/data/glue/cola/2.0.0)
INFO:absl:Constructing tf.data.Dataset glue for split validation, from gs://namratacomp8240/data/glue/cola/2.0.0
INFO:absl:Automatically caching small dataset in memory: 'all_mix:validation'
INFO:absl:Padding 'all_mix' with sequence lengths: {'inputs': 28, 'targets': 2}


INFO:tensorflow:num_cores_per_replica: 1
INFO:tensorflow:computation_shape: [1, 1, 1, 1]
INFO:tensorflow:num_replicas: 8
INFO:tensorflow:device_assignment.topology.device_coordinates: [[[0 0 0 0]
  [0 0 0 1]
  [1 0 0 0]
  [1 0 0 1]
  [0 1 0 0]
  [0 1 0 1]
  [1 1 0 0]
  [1 1 0 1]]]
INFO:tensorflow:device_assignment.core_assignment: [[[0 0 0 0]]

 [[0 0 0 1]]

 [[1 0 0 0]]

 [[1 0 0 1]]

 [[0 1 0 0]]

 [[0 1 0 1]]

 [[1 1 0 0]]

 [[1 1 0 1]]]
INFO:tensorflow:auto_logical_to_physical_tpu logical_shape=[8] physical_shape=[2, 2, 2]
INFO:tensorflow:auto_logical_to_physical_tpu logical_to_physical = [(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 1, 0), (1, 1, 1), (1, 0, 0), (1, 0, 1)]
INFO:tensorflow:SimdMeshImpl init: Shape[batch=8] LayoutRules{('vocab', 'model'), ('d_ff', 'model'), ('batch', 'batch'), ('heads', 'model'), ('experts', 'batch'), ('ensemble', 'ensemble')}
INFO:tensorflow:Device Assignment: <tensorflow.python.tpu.device_assignment.DeviceAssignment object at 0x7fce9426d6d0>
INF

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
INFO:absl:eval/all_mix/matthews_corrcoef at step 1000000: 0.000
INFO:absl:Evaluating checkpoint step: 1000000


...

INFO:absl:eval/all_mix/matthews_corrcoef at step 1010000: 17.577
INFO:absl:Evaluating checkpoint step: 1010000


INFO:tensorflow:Using config: {'_model_dir': 'gs://namratacomp8240/models/small', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.72.96.82:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.72.96.82:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.72.96.82:8470', '_evaluation_master': 'grpc://10.72.96.82:8470', '_is_chief':

INFO:absl:Load dataset info from gs://namratacomp8240/data/glue/cola/2.0.0
INFO:absl:Reusing dataset glue (gs://namratacomp8240/data/glue/cola/2.0.0)
INFO:absl:Constructing tf.data.Dataset glue for split validation, from gs://namratacomp8240/data/glue/cola/2.0.0
INFO:absl:Automatically caching small dataset in memory: 'all_mix:validation'
INFO:absl:Padding 'all_mix' with sequence lengths: {'inputs': 28, 'targets': 2}


INFO:tensorflow:num_cores_per_replica: 1
INFO:tensorflow:computation_shape: [1, 1, 1, 1]
INFO:tensorflow:num_replicas: 8
INFO:tensorflow:device_assignment.topology.device_coordinates: [[[0 0 0 0]
  [0 0 0 1]
  [1 0 0 0]
  [1 0 0 1]
  [0 1 0 0]
  [0 1 0 1]
  [1 1 0 0]
  [1 1 0 1]]]
INFO:tensorflow:device_assignment.core_assignment: [[[0 0 0 0]]

 [[0 0 0 1]]

 [[1 0 0 0]]

 [[1 0 0 1]]

 [[0 1 0 0]]

 [[0 1 0 1]]

 [[1 1 0 0]]

 [[1 1 0 1]]]
INFO:tensorflow:auto_logical_to_physical_tpu logical_shape=[8] physical_shape=[2, 2, 2]
INFO:tensorflow:auto_logical_to_physical_tpu logical_to_physical = [(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 1, 0), (1, 1, 1), (1, 0, 0), (1, 0, 1)]
INFO:tensorflow:SimdMeshImpl init: Shape[batch=8] LayoutRules{('vocab', 'model'), ('d_ff', 'model'), ('batch', 'batch'), ('heads', 'model'), ('experts', 'batch'), ('ensemble', 'ensemble')}
INFO:tensorflow:Device Assignment: <tensorflow.python.tpu.device_assignment.DeviceAssignment object at 0x7fce93895ed0>
INF

INFO:absl:eval/all_mix/matthews_corrcoef at step 1010000: 17.577
